In [1]:
#Cargar librerias
import pathlib as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
DATASET_PATH = "..\data\Acelerometro"
dataset_directory = pl.Path(DATASET_PATH)
dataset_files = dataset_directory.glob("*.csv")
dfs = []
for file in dataset_files:
    dfs.append(pd.read_csv(file))
df = pd.concat(dfs, ignore_index=True)
df.head()

,timestamp,accX,accY,accZ,label
0,0,-0.4,2.2,8.4,bajar escaleras
1,16,0.0,0.6,9.6,bajar escaleras
2,32,-1.0,1.6,10.1,bajar escaleras
3,48,-1.0,1.9,9.9,bajar escaleras
4,64,-1.3,2.2,9.3,bajar escaleras


In [5]:
df.shape

(6250, 5)

In [6]:
unique_labels = df['label'].unique()

# print the unique categories
for label in unique_labels:
    print(label)

bajar escaleras
caminar
correr
cunchillas
